In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%config Completer.use_jedi = False

In [4]:
K=10
SEED=1234

В данном ноутбуке приведен пример использования гибридной модели LightFM, включая следующие этапы:

    1) Загрузка данных
    2) Предобоработка признаков в pyspark
    3) Подбор параметров и обучение LightFM без использования признаков
    4) Подсчет метрик
    5) Подбор параметров и обучение LightFM с признаками

# 1) Загрузка данных

Пакет rs_datasets позволяет скачивать датасеты из интернета и ставится отдельно. В альфе работать не будет. 
Здесь для примера взят большой датасет, но можно взять movielens 1m, данные которого выложены в experiments/data, пример использования можно увидеть в ноутбуку replay_basics.

In [6]:
from rs_datasets import MovieLens

data = MovieLens("10m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392



items


,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance



tags


,user_id,item_id,tag,timestamp
0,15,4973,excellent!,1215184630
1,20,1747,politics,1188263867
2,20,1747,satire,1188263867


### Конвертация в формат библиотеки

In [7]:
from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "rating",
        "timestamp": "timestamp"
    }
)

### Разбиение данных

In [9]:
from replay.splitters import UserSplitter

user_random_splitter = UserSplitter(
    item_test_size=K,
    user_test_size=500,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=SEED
)

In [10]:
train, test = user_random_splitter.split(log)
train.count(), test.count()

(9995054, 5000)

In [11]:
train_opt, val_opt = user_random_splitter.split(train)
train_opt.count(), val_opt.count()

(9990054, 5000)

# 2) Предобоработка признаков в pyspark

### Конвертация в формат библиотеки

In [12]:
%%time
item_features = DataPreparator().transform(
    data=data.items,
    columns_names={
        "item_id": "item_id"
    }
)

CPU times: user 47.4 ms, sys: 0 ns, total: 47.4 ms
Wall time: 177 ms


In [13]:
item_features.show(2)

+-------+--------------------+----------------+
|item_id|              genres|           title|
+-------+--------------------+----------------+
|      1|Adventure|Animati...|Toy Story (1995)|
|      2|Adventure|Childre...|  Jumanji (1995)|
+-------+--------------------+----------------+
only showing top 2 rows



### Год

In [14]:
from pyspark.sql import functions as sf
from pyspark.sql.types import IntegerType

In [15]:
year = item_features.withColumn('year', sf.substring(sf.col('title'), -5, 4).astype(IntegerType())).select('item_id', 'year')
year.show(2)

+-------+----+
|item_id|year|
+-------+----+
|      1|1995|
|      2|1995|
+-------+----+
only showing top 2 rows



### Жанры

In [16]:
from replay.session_handler import State
from pyspark.sql.functions import split

genres = (
    State().session.createDataFrame(data.items[["item_id", "genres"]])
    .select(
        "item_id",
        split("genres", "\|").alias("genres")
    )
)

In [17]:
genres.show()

+-------+--------------------+
|item_id|              genres|
+-------+--------------------+
|      1|[Adventure, Anima...|
|      2|[Adventure, Child...|
|      3|   [Comedy, Romance]|
|      4|[Comedy, Drama, R...|
|      5|            [Comedy]|
|      6|[Action, Crime, T...|
|      7|   [Comedy, Romance]|
|      8|[Adventure, Child...|
|      9|            [Action]|
|     10|[Action, Adventur...|
|     11|[Comedy, Drama, R...|
|     12|    [Comedy, Horror]|
|     13|[Animation, Child...|
|     14|             [Drama]|
|     15|[Action, Adventur...|
|     16|      [Crime, Drama]|
|     17|[Comedy, Drama, R...|
|     18|[Comedy, Drama, T...|
|     19|            [Comedy]|
|     20|[Action, Comedy, ...|
+-------+--------------------+
only showing top 20 rows



In [18]:
from pyspark.sql.functions import explode

genres_list = (
    genres.select(explode("genres").alias("genre"))
    .distinct().filter('genre <> "(no genres listed)"')
    .toPandas()["genre"].tolist()
)

In [19]:
genres_list

['Documentary',
 'Fantasy',
 'IMAX',
 'Adventure',
 'War',
 'Animation',
 'Comedy',
 'Thriller',
 'Film-Noir',
 'Crime',
 'Sci-Fi',
 'Musical',
 'Mystery',
 'Drama',
 'Horror',
 'Western',
 'Romance',
 'Children',
 'Action']

In [20]:
from pyspark.sql.functions import col, lit, array_contains
from pyspark.sql.types import IntegerType

item_features = genres
for genre in genres_list:
    item_features = item_features.withColumn(
        genre,
        array_contains(col("genres"), genre).astype(IntegerType())
    )
item_features = item_features.drop("genres").cache()
item_features.count()

10681

In [21]:
item_features.show(2)

+-------+-----------+-------+----+---------+---+---------+------+--------+---------+-----+------+-------+-------+-----+------+-------+-------+--------+------+
|item_id|Documentary|Fantasy|IMAX|Adventure|War|Animation|Comedy|Thriller|Film-Noir|Crime|Sci-Fi|Musical|Mystery|Drama|Horror|Western|Romance|Children|Action|
+-------+-----------+-------+----+---------+---+---------+------+--------+---------+-----+------+-------+-------+-----+------+-------+-------+--------+------+
|      1|          0|      1|   0|        1|  0|        1|     1|       0|        0|    0|     0|      0|      0|    0|     0|      0|      0|       1|     0|
|      2|          0|      1|   0|        1|  0|        0|     0|       0|        0|    0|     0|      0|      0|    0|     0|      0|      0|       1|     0|
+-------+-----------+-------+----+---------+---+---------+------+--------+---------+-----+------+-------+-------+-----+------+-------+-------+--------+------+
only showing top 2 rows



In [22]:
item_features = item_features.join(year, on='item_id', how='inner')
item_features.count()

10681

In [23]:
item_features.cache()

DataFrame[item_id: int, Documentary: int, Fantasy: int, IMAX: int, Adventure: int, War: int, Animation: int, Comedy: int, Thriller: int, Film-Noir: int, Crime: int, Sci-Fi: int, Musical: int, Mystery: int, Drama: int, Horror: int, Western: int, Romance: int, Children: int, Action: int, year: int]

# 3) Подбор параметров и обучение LightFM без использования признаков

In [24]:
from replay.models import LightFMWrap

# зафиксируем лосс-функцию - warp loss, так как зачастую он работает довольно хорошо
model = LightFMWrap(random_state=SEED, loss='warp')

Посмотрим на используемые по умолчанию границы параметров для поиска:

In [25]:
model._search_space

{'loss': {'type': 'categorical',
  'args': ['logistic', 'bpr', 'warp', 'warp-kos']},
 'no_components': {'type': 'loguniform_int', 'args': [8, 512]}}

Будем искать оптимальное число компонент

In [87]:
best_params = model.optimize(train=train_opt, test=val_opt, param_grid={'no_components':[5, 128]}, budget=10)

24-May-21 11:14:13, replay, DEBUG: Фит модели в оптимизации
DEBUG:replay:Фит модели в оптимизации
24-May-21 11:14:13, replay, DEBUG: Начало обучения LightFMWrap
DEBUG:replay:Начало обучения LightFMWrap
24-May-21 11:14:13, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
24-May-21 11:16:22, replay, DEBUG: Предикт модели в оптимизации
DEBUG:replay:Предикт модели в оптимизации
24-May-21 11:16:22, replay, DEBUG: Начало предикта LightFMWrap
DEBUG:replay:Начало предикта LightFMWrap
24-May-21 11:17:21, replay, DEBUG: Подсчет метрики в оптимизации
DEBUG:replay:Подсчет метрики в оптимизации
24-May-21 11:17:44, replay, DEBUG: NDCG=0.140141
DEBUG:replay:NDCG=0.140141
[I 2021-05-24 11:17:44,106] Trial 1 finished with value: 0.14014071281428445 and parameters: {'loss': 'warp', 'no_components': 6}. Best is trial 1 with value: 0.14014071281428445.
24-May-21 11:17:44, replay, DEBUG: Фит модели в оптимизации
DEBUG:replay:Фит модели в оптимизации
24-May-21 11:17:

DEBUG:replay:Начало обучения LightFMWrap
24-May-21 11:49:03, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
24-May-21 11:51:53, replay, DEBUG: Предикт модели в оптимизации
DEBUG:replay:Предикт модели в оптимизации
24-May-21 11:51:53, replay, DEBUG: Начало предикта LightFMWrap
DEBUG:replay:Начало предикта LightFMWrap
24-May-21 11:52:53, replay, DEBUG: Подсчет метрики в оптимизации
DEBUG:replay:Подсчет метрики в оптимизации
24-May-21 11:53:15, replay, DEBUG: NDCG=0.184472
DEBUG:replay:NDCG=0.184472
[I 2021-05-24 11:53:15,715] Trial 10 finished with value: 0.18447151841981346 and parameters: {'loss': 'warp', 'no_components': 35}. Best is trial 8 with value: 0.1924041088075205.


In [26]:
best_params = {'no_components': 63}

In [27]:
best_params

{'no_components': 63}

In [28]:
model = LightFMWrap(random_state=SEED, loss='warp', **best_params)

In [29]:
%%time
model.fit(train)

24-May-21 13:55:23, replay, DEBUG: Начало обучения LightFMWrap
DEBUG:replay:Начало обучения LightFMWrap
24-May-21 13:55:23, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
24-May-21 13:55:27, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)


CPU times: user 31min 25s, sys: 6.92 s, total: 31min 32s
Wall time: 1min 17s


In [30]:
%%time
recs = model.predict(
    k=K,
    users=test.select('user_id').distinct(),
    log=train,
    filter_seen_items=True
)

24-May-21 13:56:40, replay, DEBUG: Начало предикта LightFMWrap
DEBUG:replay:Начало предикта LightFMWrap


CPU times: user 6.25 s, sys: 1.39 s, total: 7.64 s
Wall time: 26.7 s


# 4) Подсчет метрик

In [31]:
from replay.metrics import HitRate, NDCG, MAP
from replay.experiment import Experiment

metrics = Experiment(test, {NDCG(): K,
                            MAP() : K,
                            HitRate(): [1, K]})


In [32]:
%%time
metrics.add_result("LightFM_no_features", recs)
metrics.results

CPU times: user 74.8 ms, sys: 20.1 ms, total: 94.9 ms
Wall time: 14.3 s


,HitRate@1,HitRate@10,MAP@10,NDCG@10
LightFM_no_features,0.38,0.82,0.126265,0.24204


# 5) Подбор параметров и обучение LightFM с использованием признаков

In [33]:
from replay.models import LightFMWrap

model_feat = LightFMWrap(random_state=SEED, loss='warp')

In [34]:
best_params = model_feat.optimize(train=train_opt, test=val_opt, param_grid={'no_components':[5, 256]}, budget=10, item_features=item_features)

[I 2021-05-24 13:57:22,270] A new study created in memory with name: no-name-8f8a850c-a8ca-43fb-b0f7-b9466b937745
24-May-21 13:57:22, replay, DEBUG: Фит модели в оптимизации
DEBUG:replay:Фит модели в оптимизации
24-May-21 13:57:22, replay, DEBUG: Начало обучения LightFMWrap
DEBUG:replay:Начало обучения LightFMWrap
24-May-21 13:57:22, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
24-May-21 13:57:25, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
24-May-21 14:02:30, replay, DEBUG: Предикт модели в оптимизации
DEBUG:replay:Предикт модели в оптимизации
24-May-21 14:02:30, replay, DEBUG: Начало предикта LightFMWrap
DEBUG:replay:Начало предикта LightFMWrap
24-May-21 14:02:59, replay, DEBUG: Подсчет метрики в оптимизации
DEBUG:replay:Подсчет метрики в оптимизации
24-May-21 14:03:05, replay, DEBUG: NDCG=0.150733
DEBUG:replay:NDCG=0.150733
[I 2021-05-24 14:03:05,912] Trial 0 finished wit

24-May-21 15:47:23, replay, DEBUG: Фит модели в оптимизации
DEBUG:replay:Фит модели в оптимизации
24-May-21 15:47:23, replay, DEBUG: Начало обучения LightFMWrap
DEBUG:replay:Начало обучения LightFMWrap
24-May-21 15:47:23, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
24-May-21 15:54:34, replay, DEBUG: Предикт модели в оптимизации
DEBUG:replay:Предикт модели в оптимизации
24-May-21 15:54:34, replay, DEBUG: Начало предикта LightFMWrap
DEBUG:replay:Начало предикта LightFMWrap
24-May-21 15:55:02, replay, DEBUG: Подсчет метрики в оптимизации
DEBUG:replay:Подсчет метрики в оптимизации
24-May-21 15:55:09, replay, DEBUG: NDCG=0.158515
DEBUG:replay:NDCG=0.158515
[I 2021-05-24 15:55:09,583] Trial 9 finished with value: 0.158514580980604 and parameters: {'no_components': 12}. Best is trial 4 with value: 0.19471973737482653.


In [39]:
best_params

{'no_components': 79}

#### best_params

In [35]:
model_feat = LightFMWrap(random_state=SEED, **best_params, loss='warp')

In [36]:
%%time
model_feat.fit(train, item_features=item_features)

24-May-21 15:55:09, replay, DEBUG: Начало обучения LightFMWrap
DEBUG:replay:Начало обучения LightFMWrap
24-May-21 15:55:09, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
24-May-21 15:55:10, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)


CPU times: user 12h 32min 47s, sys: 4min 36s, total: 12h 37min 24s
Wall time: 16min 47s


In [37]:
%%time
recs = model_feat.predict(
    k=K,
    users=test.select('user_id').distinct(),
    log=train,
    filter_seen_items=True,
    item_features=item_features
)

24-May-21 16:11:56, replay, DEBUG: Начало предикта LightFMWrap
DEBUG:replay:Начало предикта LightFMWrap


CPU times: user 6.14 s, sys: 1.39 s, total: 7.52 s
Wall time: 24.9 s


In [38]:
metrics.add_result("LightFM_item_features", recs)
metrics.results

,HitRate@1,HitRate@10,MAP@10,NDCG@10
LightFM_no_features,0.380,0.820,0.126265,0.24204
LightFM_item_features,0.288,0.772,0.088089,0.18762


Когда LightFM используется с признаками пользователей или пользователей и объектов, зачастую признаки позволяют улучшить качество рекомендаций и рекомендовать холодным пользователям. В нашем случае, с использованием только признаков объектов этого не произошло.